# Deploy COA Objects in Transcend:   DBInfo (Demo Project)

In [ ]:
%addconnect name=Transcend, user=sh186014, host=tdprdcop3.td.teradata.com, props="LOGMECH=LDAP"

In [3]:
%connect name=Transcend

Password ············


Success: 'Transcend' connection established


In [5]:
call adlste_coa.sp_meta_objects_refresh('DBCInfo') 

Success: 0 rows affected

DatabaseName,TableName,TableKind,ObjType,ObjFunction,ProcessName
ADLSTE_COA,coa_dat_dbcinfo,V,coa,dat,dbcinfo
ADLSTE_COA,coat_dat_dbcinfo,T,coat,dat,dbcinfo
ADLSTE_COA,sp_dat_dbcinfo,P,sp,dat,dbcinfo
ADLSTE_COA,stg_dat_dbcinfo,O,stg,dat,dbcinfo
ADLSTE_COA_Stg,stg_dat_dbcinfo,O,stg,dat,dbcinfo


## Stage Object (adlste_coa_stg)

In [14]:
drop table adlste_coa_stg.stg_dat_dbcinfo;

Success: 18 rows affected

In [15]:
CREATE MULTISET GLOBAL TEMPORARY TABLE adlste_coa_stg.stg_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) NO PRIMARY INDEX 
ON COMMIT PRESERVE ROWS;

Success: 0 rows affected

In [16]:
select count(*) from adlste_coa_stg.stg_dat_dbcinfo;

Count(*)
0


## Target Table and View (adlste_coa)

In [18]:
DROP TABLE adlste_coa.coat_dat_DBCinfo;

Success: 27 rows affected

In [20]:
CREATE MULTISET TABLE adlste_coa.coat_dat_DBCinfo
( Account_Name   VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, Site_ID        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, LogDate        DATE Default Current_Date
, InfoKey        VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
, InfoData       VARCHAR(128) CHARACTER SET LATIN NOT CASESPECIFIC
) Primary Index(Site_ID, LogDate);

Success: 0 rows affected

In [21]:
REPLACE VIEW adlste_coa.coa_dat_DBCinfo
as locking row for access
Select * from adlste_coa.coat_dat_DBCinfo;

Success: 9 rows affected

## Stored Proc to manage the merge

In [24]:
show procedure adlste_coa.sp_dat_DBCinfo

RequestText
REPLACE PROCEDURE adlste_coa.
sp_dat_dbcinfo
"( fileset_version VARCHAR(128) ) SQL SECURITY CREATOR BEGIN --call sp_audit_Account_Name ('adlste_coa.stg_dat_dbcinfo', 'sp_dat_dbcinfo', '{}'); --"


In [40]:
REPLACE PROCEDURE adlste_coa.sp_dat_dbcinfo ( fileset_version VARCHAR(128) ) 
SQL SECURITY CREATOR
BEGIN 
  --call sp_audit_Account_Name ('adlste_coa.stg_dat_dbcinfo', 'sp_dat_dbcinfo', '{}'); 
  --call sp_audit_Site_ID ('adlste_coa.stg_dat_dbcinfo', 'sp_dat_dbcinfo', '{}'); 


  DELETE FROM adlste_coa.coat_dat_dbcinfo WHERE (Site_ID, LogDate) IN
   (Select Site_ID, Current_Date From adlste_coa_stg.stg_dat_dbcinfo );
  
  Insert into adlste_coa.coat_dat_dbcinfo 
  Select Account_Name, Site_ID, Current_Date as LogDate, InfoKey, InfoData 
  from adlste_coa_stg.stg_dat_dbcinfo
  group  by 1,2,3,4,5; 

END;

Success: 0 rows affected

In [41]:
call adlste_coa.sp_dat_dbcinfo('1.1')

Success: 0 rows affected

## Test the Process

In [42]:
insert into adlste_coa_stg.stg_dat_dbcinfo 
Select 'Test_Account',  'Test_SiteID', InfoKey, InfoData from dbc.dbcinfo;

Select * from adlste_coa_stg.stg_dat_dbcinfo 

Success: 3 rows affected

Account_Name,Site_ID,InfoKey,InfoData
Test_Account,Test_SiteID,RELEASE,16.20.32.11
Test_Account,Test_SiteID,VERSION,16.20.32.11
Test_Account,Test_SiteID,LANGUAGE SUPPORT MODE,Standard
Test_Account,Test_SiteID,RELEASE,16.20.32.11
Test_Account,Test_SiteID,VERSION,16.20.32.11
Test_Account,Test_SiteID,LANGUAGE SUPPORT MODE,Standard
Test_Account,Test_SiteID,RELEASE,16.20.32.11
Test_Account,Test_SiteID,VERSION,16.20.32.11
Test_Account,Test_SiteID,LANGUAGE SUPPORT MODE,Standard


In [43]:
call adlste_coa.sp_dat_dbcinfo('1.1');

Success: 0 rows affected

In [44]:
Select * from adlste_coa.coa_dat_dbcinfo 

Account_Name,Site_ID,LogDate,InfoKey,InfoData
Test_Account,Test_SiteID,2020-05-06,LANGUAGE SUPPORT MODE,Standard
Test_Account,Test_SiteID,2020-05-06,RELEASE,16.20.32.11
Test_Account,Test_SiteID,2020-05-06,VERSION,16.20.32.11
